In [1]:
import sqlite3
import re
from bs4 import BeautifulSoup
import os
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
con = sqlite3.connect('slovari.db')
cur = con.cursor()

In [4]:
#cur.execute('CREATE TABLE test (id INTEGER PRIMARY KEY, orth VARCHAR(30), phon VARCHAR(30), sense_n INTEGER, sense VARCHAR(3000), examp VARCHAR(3000), etym VARCHAR(30), syn VARCHAR(30), ant VARCHAR(30), epith VARCHAR(30), pos VARCHAR(10), gender VARCHAR(4), asp VARCHAR(10), gov VARCHAR(10), dic_name VARCHAR(30))')
#con.commit()

In [4]:
def get_dicname(file):
    res = re.findall('<title>.+?</title>', file) 
    for item in res:
        item = item.replace('<title>', '')
        item = item.replace('</title>', '')
    return item

In [5]:
to_replace = ["['> ", " <']", "['>", "<']", '["> ', ' <"]', '[">', '<"]', "['", "']"]

def strip_all(to_replace, line_with_tags):
    line = str(re.findall('>.+?<', line_with_tags))
    for item_to_replace in to_replace:
        line = line.replace(item_to_replace, "")
    if "..." in line:
        line = line.replace("...", "")
    return line

In [6]:
# ! extent
def get_orth(entry):
    res = re.findall('<orth extent="full" type="lemma">.+?</orth>', entry)
    if len(res) != 0:
        for item in res:
            item = strip_all(to_replace, item)
            item = item.lower()
    else:
        res = re.findall('<orth extent="part" type="lemma">.+?</orth>', entry)
        for item in res:
            item = strip_all(to_replace, item)
            item = item.lower()
    try:
        return item
    except:
        pass

In [7]:
# ! type
def get_phon(entry):
    res = re.findall('<phon type="main">.+?</phon>', entry)
    for item in res:
        item = strip_all(to_replace, item)
    try:
        return item
    except:
        pass

In [8]:
to_replace_in_sense = ["'", '"', '[', ']', 'n', '=']

def get_sense_n(to_replace_in_sense, entry):
    res = re.findall('<sense n=".">.+?</sense>', entry)
    n_senses = []
    for item in res:
        n_sense = str(re.findall('n="\d"', item))
        for item_to_replace in to_replace_in_sense:
            n_sense = n_sense.replace(item_to_replace, '')
        n_senses.append(int(n_sense))
    try:
        return n_senses
    except:
        pass

In [9]:
# ! usg
def get_sense(num, entry, tag):
    sense = str(re.findall('<sense n="' + str(num) + '">.+?</sense>', entry))
    res = str(re.findall('<' + tag + '>.+?</' + tag + '>', sense))
    res = res.replace("['<" + tag + ">" +" " + "<text>", "")
    res = res.replace("</text></" + tag + ">']", "")
    if res != "[]":
        return res

In [10]:
# ! язык (от фран. - фран.), чтобы можно было найти "все слова, заимствованные из фран." 
def get_etym(entry):
    res = re.findall('<etym><text>.+</text></etym>', entry)
    for item in res:
        item = item.replace('<etym>', '')
        item = item.replace('</etym>', '')
        item = strip_all(to_replace, item)
    try:
        return item
    except:
        pass

In [11]:
def get_syn(entry):
    res = re.findall('<xr type="synonym"><ref>.+</ref>', entry)
    for item in res:
        item = item.replace('<xr type="synonym">', '')
        item = item.replace('<ref>', '')
        item = item.replace('</ref>', '')
    try:
        return item
    except:
        pass

In [12]:
def get_ant(entry):
    res = re.findall('<xr type="antonym"><ref>.+</ref></xr>', entry)
    for item in res:
        item = item.replace('<xr type="antonym">', '')
        item = item.replace('</xr>', '')
        item = strip_all(to_replace, item)
    try:
        return item
    except:
        pass

In [13]:
# ! авторы эпитетов (src)
def get_epith(entry):
    res = re.findall('<xr type="epithet"><ref>.+?</ref>', entry)
    for item in res:
        item = item.replace('<xr type="epithet">', '')
        item = item.replace('<ref>', '')
        item = item.replace('</ref>', '')
    try:
        return item
    except:
        pass

In [14]:
def get_epith_into_sense(entry, num):
    sense = str(re.findall('<sense n="' + str(num) + '">.+?</sense>', entry))
    res = re.findall('<xr type="epithet"><ref>.+?</ref>', sense)
    for item in res:
        item = item.replace('<xr type="epithet">', '')
        item = item.replace('<ref>', '')
        item = item.replace('</ref>', '')
    try:
        return item
    except:
        pass

In [15]:
# ! inflection
def get_gram_into_form(entry, tag):
    form = str(re.findall('<form>.+?</form>', entry))
    res = str(re.findall('<' + tag + '>.+?</' + tag + '>', form))
    res = res.replace("['<" + tag + ">", "")
    res = res.replace("</" + tag + ">']", "")
    if res != "[]":
        return res

In [16]:
def get_gram_into_sense(num, entry, tag):
    sense = str(re.findall('<sense n="' + str(num) + '">.+?</sense>', entry))
    res = str(re.findall('<' + tag + '>.+?</' + tag + '>', sense))
    res = res.replace("['<" + tag + ">", "")
    res = res.replace("</" + tag + ">']", "")
    if res != "[]":
        return res

In [17]:
path = "C:/Users/1/Desktop/dics/"
to_replace_in_file = ['\t', '\n']

In [24]:
for fname in os.listdir(path):
    
    file_path = path + fname
    file = open(file_path, "r", encoding="utf-8")
    file = file.read()
    dic_name = get_dicname(file) 
    for item in to_replace_in_file:
        file = file.replace(item, '')
    soup = BeautifulSoup(file)
    entries = soup.findAll('entry')
    
    for entry in entries:
        entry = str(entry)
        entry = entry.replace("\xa0", " ")
        orth = get_orth(entry)
        epith = get_epith(entry)
        num_senses = get_sense_n(to_replace_in_sense, entry)
        if len(num_senses) == 0:
            cur.execute('INSERT INTO test (orth, epith, dic_name) VALUES (:orth, :epith, :dic_name)', (orth, epith, dic_name))
            con.commit()
        else:
            for num in num_senses:
                sense = get_sense(num, entry, 'def')
                epith = get_epith_into_sense(entry, num)
                cur.execute('INSERT INTO test (orth, sense_n, sense, epith, dic_name) VALUES (:orth, :sense_n, :sense, :epith, :dic_name)', (orth, num, sense, epith, dic_name))
con.commit()

In [25]:
con.close()